# A Crash Course in Applied Linear Algebra
Patrick Landreman | Spry Health

### Some Motivation: Predicting the Future

Since we are all busy people with too much to do, why should you spend your precious time reading this notebook? I'd like to give you a sample of the kind of power you'll have by the end.

Consider the following time series plot:

<img src='img/intro_system_ringing.png'>

This could be the value of a stock price, or the position of an aircraft, or some biological signal, or any number of things. You are tasked with predicting the value at some future time. Looks unpleasant, no? Maybe there is some periodicity emerging by the end, but surely this is not a trivial task.

Now suppose you know that the series above was a mixture of a number of much simpler functions, and you could tell the amount of each function:

<img src='img/intro_system_modes.png'>

Predicting how each of these functions will evolve should be a much simpler task. The question is - how do you find these simpler functions, and how do you find the weight of each function?

Perhaps more interesting, suppose the system above comes from a robotic arm which can move in a 2D plane. The time series corresponds to the position of the end of the arm in one direction - we kicked the system, which caused some shaking and oscillating that subsides over time. 

We'll describe the complete position with a pair of numbers, $(y_1, y_2)$. We also can control our robot with some inputs, $(u_1, u_2)$. If we wanted to drive the arm to a specific location, say $(y_1, y_2) = (0.2, -0.2)$, we can expect that the arm will move, and then because of momentum or other issues the system might overshoot a bit and take some time to stabilize (this is exactly the kind of thing that happens with the read head in a spinning-disk hard drive, for instance). The next image shows what happens if at 50 seconds we apply a specific set of inputs to reach the target:

<img src='img/intro_naive_control.png'>

Ok, we get there after ~3000 seconds, but it takes a while for the ringing to settle down.

Instead, however, it is possible to choose a more complicated sequence of inputs that brings the arm to the target in **exactly** a specified amount of time:

<img src='img/intro_least_norm_control.png'>

Here, the arm reaches the specified coordinates at 800 seconds and remains there, perfectly stable. From looking at the sequence of inputs, there is **no way** a human could intuit the sequence.

This is just one of the kinds of problems I hope to expose you to today.

### Acknowledgements

*This presentation is dedicated to Prof. Stephen Boyd and Prof. Sanjay Lall, who first exposed me to this way of seeing the world. Many of the ideas I present here were inspired largely by them and their teachers, and they deserve credit for the enormous amount of work they have done.*

*If this subject interests you, I* strongly *suggest exploring their [books](http://vmls-book.stanford.edu/), [course notes](http://ee263.stanford.edu/), and lecture series on [YouTube](https://www.youtube.com/playlist?list=PL06960BA52D0DB32B).*

### Python Setup

Everything in this talk can be done with a basic installation of Numpy and Scipy. The version should not be important. Scipy is used exclusively for some convenience functions, and Matplotlib is included only for visualization purposes. Neither are necessary for linear algebra. This notebook was written using Python 3.6, Numpy 1.16.4, Scipy 1.2.1 and Matplotlib 3.1.1

In [9]:
# Render MPL figures within notebook cells
%matplotlib inline

# Import python libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [10]:
# Configure some defaults for plots
rcParams['font.size'] = 16
rcParams['figure.figsize'] = (10, 3)

In [11]:
# Set Numpy's random number generator so the same results are produced each time the notebook is run
np.random.seed(0)

### Why did I make this notebook?


If your exposure to the linear algebra was anything like my first course, you probably don't think back on the experience longingly, wishing you could relive those days. My professor literally opened the class by saying he didn't understand why linear algebra was interesting or useful. It was a pretty painful semester, full of abstract concepts like vector subspaces, and I just couldn't connect with the material. It wasn't until the end of my graduate school career when I took a class that beat me over the head with examples of how powerful and magical this subject can be. I want to share that experience with more people.

Associating linear algebra with a kind of fascination or awe is especially helpful, because things like matrices and vectors have a way of showing up in just about every quantitative situation. In my world of signal processing and statistics, it's virtually impossible to get through a research paper without being comfortable manipulating vectors and matrices. 

Where the magic really becomes clear is when - for whatever discipline you're working in - you manage to cram your problem into something that looks like 

<br>

<center>
$
\begin{equation*}
y = Ax
\end{equation*}
$
</center>
<br>

suddenly you can apply ideas from finance, quantum mechanics, operations research, RADAR, medical imaging, or any number of other fields to solve your problem. The only thing you need to learn to be dangerous in any of these fields is an intution for the properties and structure of the matrix $A$.

So how can four ASCII characters (not including the spaces) apply to so many situations in the real world almost stupidly well? In part because each of these symbols encompass a great deal of complexity. Let's start by unpacking them one by one:

### Vectors and Matrices

All of this should be material you've seen in courses before, so I'll go through it very quickly just to put us on the same footing as far as notation.

Beginning with $y$:

$y$ is a **vector**, which is essentially a list of numbers. It's often useful to think of vectors as arrows that point from the origin to some point in space, where each entry of $y$ corresponds to a dimension in that space. For a 2-dimensional example, we could have a vector with entries $[2, 3]$ that one could visualize as 

<img src='img/vector.png' style='height: 250px'>

As convention, lowercase variables without subscripts will be vectors. If $y$ is a vector, then $y_1$, $y_2 \ldots$ are the entries of $y$.

Vectors add elementwise - that is, you add the elements in the corresponding positions. Graphically, this has the effect of starting one vector from the point of the other vector. The vector sum is the final point that is reached:

<img src='img/vector_addition.png' style='height: 250px'>


Vectors can be written in row or column form. For this presentation, all vectors are column vectors, and the corresponding row vectors are notated as e.g. $y^\mathsf{T}$. The inner product (or dot product) of two vectors is 

<center>
$y^\mathsf{T} x=
\begin{bmatrix}
y_1 & y_2 & \ldots & y_n
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{bmatrix}
= x_1y_1 + x_2y_2 + \ldots + x_ny_n$
</center>

(This is a scalar)

$A$ is a **matrix**. Matrices are a grid of numbers, and can be thought of as a horizontal stack of column vectors (or a vertical stack of row vectors). Matrices add elementwise. Multiplying a matrix by a vector is equivalent to a batch inner product of the vector with the rows of $A$.

I'll use uppercase variables to indicate matrices, and lowercase letters with two subscripts, e.g. $a_{i,j}$, indicate the element in the $i$th row and $j$th column of $A$. If you encounter a matrix element with only one subscript, such as $a_i$, this refers to the vector containing the $i$th column of $A$. A lowercase character with a tilde, e.g. $\tilde{a}_j$ means the $j$th row of $A$ *as a column vector*.

In Python, we'll use Numpy arrays for all of our matrices and vectors:

In [12]:
m, n = 3, 2

# vectors are row vectors by default in Numpy
y = np.arange(m)
y

array([0, 1, 2])

In [13]:
# To get column vectors we need to do a reshape operation
y = y.reshape(-1, 1)
y

array([[0],
       [1],
       [2]])

In [14]:
# Matrices are defined from lists or Numpy's construction tools
A = np.array([[1, 2, 3],
              [4, 5, 6]])
A

array([[1, 2, 3],
       [4, 5, 6]])

In [15]:
B = np.ones((m, n))
B

array([[1., 1.],
       [1., 1.],
       [1., 1.]])

### Where do matrices come from?

A system of linear equations

<center>
$
\begin{align*}
    y_1 &= a_{1,1}x_1 + a_{1,2}x_2 + a_{1,3}x_3 + \ldots + a_{1,n}x_{n} \\
    y_2 &= a_{2,1}x_1 + a_{2,2}x_2 + a_{2,3}x_3 + \ldots + a_{2,n}x_{n} \\
    \vdots \\
    y_m &= a_{m,1}x_1 + a_{m,2}x_2 + a_{m,3}x_3 + \ldots + a_{m,n}x_{n} \\
\end{align*}
$
</center>
  
can be re-written as 
    
<center>
$
\begin{bmatrix}
y_1  \\
y_2  \\
\vdots  \\
y_m
\end{bmatrix}
=
\begin{bmatrix}
a_{1,1} & a_{1,2} & a_{1,3} & \ldots & a_{1,m} \\
a_{2,1} & a_{2,2} & a_{2,3} & \ldots & a_{2,m} \\
\vdots \\
a_{n,1} & a_{n,2} & a_{n,3} & \ldots & a_{n,m} \\
\end{bmatrix}
\begin{bmatrix}
x_1  \\
x_2  \\
\vdots  \\
x_m
\end{bmatrix}
$
</center>

which we simplify as

<center>
$y=Ax$
</center>


We say that 

$y \in \mathbb{R}^m$  
$x \in \mathbb{R}^n$  
$A \in \mathbb{R}^{m \times n}$

...but of course these could also be complex.

I'll stick to this convention of using $m$ as the number of rows and $n$ as the number of columns everywhere.

### Examples

#### Classical Mechanics

<img src='img\rocket.png' style='height: 300px'>

* $x$ represents a series of forces being applied at different points
* $y$ represents the net force and torque on the total body
* $\mathbf{A}$ is determined by the geometry of the system

#### Buisness Operations

<img src='img\excel.jpg' style='height: 150px'>

e.g. Cost of production

* $x$ represents the cost per component
* $y$ represents the cost per finished product
* $\mathbf{A}$ is the Bill of Materials for each product

#### Statistics

e.g. Autoregressive Models

<br>
<center>
$
\begin{align*}
    x(T) &= a_1x(T-1) + a_2x(T-2) + a_3x(T-3) + \ldots + a_nx(T-n) \\
\end{align*}
$
</center>

<br>
<br>

<center>
$
\begin{bmatrix}
x(T+1)  \\
\vdots  \\
x(n+2)  \\
x(n+1)  \\
\end{bmatrix}
=
\begin{bmatrix}
x(T-n+1) & x(T-n+2) & x(T-n+3) & \ldots & x(T) \\
\vdots \\
x(2) & x(3) & x(4) & \ldots & x(n+1) \\
x(1) & x(2) & x(3) & \ldots & x(n) \\
\end{bmatrix}
\begin{bmatrix}
a_1  \\
a_2  \\
\vdots  \\
a_n
\end{bmatrix}
$
</center>


* $x$ is the coefficients of the model for each lag
* $y$ is the time series output of the model
* $\mathbf{A}$ is a stack of lagged windows of the time series

### More than just Lines

This last example of an Autoregressive Model, as well as the time series forecasting example at the beginning, are great for highlighting that linear algebra applies to situations that look anything but linear! Let's explore this idea more closely - consider the case of a polynomial evaluated at $m$ different points:

<br>

<center>
$
\begin{align*}
    y_1 &= a_1 + a_{2}x_1 + a_{3}x_1^2 + \ldots + a_{n}x_1^{n-1} \\
    y_2 &= a_1 + a_{2}x_2 + a_{3}x_2^2 + \ldots + a_{n}x_2^{n-1} \\
    \vdots \\
    y_m &= a_1 + a_{2}x_m + a_{3}x_m^2 + \ldots + a_{n}x_m^{n-1} \\
\end{align*}
$
</center>

<br>

It's certainly true that these equations are not linear as far as $x$ is concerned. BUT! The coefficients of the polynomial are all first order, which means we can write the system of equations this way:
<br>

<center>
$
\begin{bmatrix}
y_1  \\
y_2  \\
\vdots  \\
y_m  \\
\end{bmatrix}
=
\begin{bmatrix}
1 & x_1 & x_1^2 & \ldots & x_1^{n-1} \\
1 & x_2 & x_2^2 & \ldots & x_2^{n-1} \\
\vdots \\
1 & x_m & x_m^2 & \ldots & x_m^{n-1} \\
\end{bmatrix}
\begin{bmatrix}
a_1  \\
a_2  \\
\vdots  \\
a_n
\end{bmatrix}
$
</center>


In fact, there is absolutely no requirement on where the entries of $A$ come from! They often will arise from some horrendous expressions involving sines, logarithms, or enhanced spherical Riemann functions of the 60th kind (I made that up). As long as you can remain calm, slowly step back, and recognize that the **mixture** of these functions is still linear, then you still have

<br>

<center>
$
\begin{align*}
    y_1 &= a_{1}f_1(x_1) + a_{2}f_2(x_1) + a_{3}f_3(x_1) + \ldots + a_{n}f_n(x_1) \\
    y_2 &= a_{1}f_1(x_2) + a_{2}f_2(x_2) + a_{3}f_3(x_2) + \ldots + a_{n}f_n(x_2) \\
    \vdots \\
    y_m &= a_{1}f_1(x_m) + a_{2}f_2(x_m) + a_{3}f_3(x_m) + \ldots + a_{n}f_n(x_m) \\
\end{align*}
$
</center>

<br>
<br>

<center>
$
\begin{bmatrix}
y_1  \\
y_2  \\
\vdots  \\
y_m  \\
\end{bmatrix}
=
\begin{bmatrix}
f_1(x_1) & f_2(x_1) & f_3(x_1) & \ldots & f_n(x_1) \\
f_1(x_2) & f_2(x_2) & f_3(x_2) & \ldots & f_n(x_2) \\
\vdots \\
f_m(x_m) & f_2(x_m) & f_3(x_m) & \ldots & f_n(x_m) \\
\end{bmatrix}
\begin{bmatrix}
a_1  \\
a_2  \\
\vdots  \\
a_n
\end{bmatrix}
$
</center>

#### Linearization

Another reason why linear algebra applies to so many situations is that many problems which are *technically* nonlinear can be approximated as linear with a negligible amount of error. A common way this comes up (and a favorite approach of physicists everywhere) is by invoking a "Taylor series approximation". Imagine you are standing at a point $a$ on a number line, and you know the value of some function $f$ at your current location. If you were to move from $a$ to a new point $x$, the change you would see in the value of the function is a weighted sum of different derivatives of $f$:

$f(x) = f(a)+\frac {f'(a)}{1!} (x-a)+ \frac{f''(a)}{2!} (x-a)^2+\frac{f'''(a)}{3!}(x-a)^3+ \cdots$  

The weighting (read: "practical importance") of the terms depends on how far you have moved. For any function, in any practical situation, there will be some point close to $a$ where the error from throwing away all the terms which are nonlinear in $x$ is irrelevant to what you are trying to accomplish.

That is, for $x$ "very" close to $a$:  
  
$f(x) \sim f(a)+\frac {f'(a)}{1!} (x-a)$

It can be helpful to see this visually. The figure below shows Taylor approximations to $f(x)=\sin(x)$ for $a=0$. The order of the approximations are 1, 3, 5, 7, 9, 11 and 13. Notice how in the range (-1, 1), a straight line looks reasonably equivalent to $\sin(x)$! Whether or not it is "reasonable" depends on your application.

<img src='img/taylor_series.png' style='height:300px'>



#### Example - GPS

<img src='img/gps.png' style='height: 300px'>

GPS sattelites orbit at an altitude of ~20 km<sup>1</sup>. For a 1-meter deviation off-axis, the resulting fractional error from linearizing would be 

In [16]:
x = 20e3
y = 1

(np.sqrt(x**2 + y**2) - x) / x

1.2500000593718141e-09

Assuming that you can live with a few nano-percent of error in locating your position, then GPS may as well be a linear problem.


<sup>1</sup>https://www.gps.gov/systems/gps/space/